In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00


In [3]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296183 sha256=0e765a80886efb1f930278eba6c809f5d25d6535de10e1508a853106acb2eafb
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [2]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import fasttext
from scipy.stats import uniform

# Mount Google Drive
drive.mount('/content/drive')

# Download and extract the FastText model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz # Downloads the pre-trained model
!gunzip cc.en.300.bin.gz # Extracts the downloaded model

# Function to convert "ANSWER" column to FastText embeddings and return the dataset
def convert_to_fasttext_embeddings(input_file, text_column):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Load pre-trained FastText model
    model = fasttext.load_model('cc.en.300.bin')

    # Function to get FastText embeddings for a text
    def get_fasttext_embeddings(text):
        # Generate the word vector for the entire text by averaging word vectors
        words = text.split()
        word_vectors = [model.get_word_vector(word) for word in words]
        return np.mean(word_vectors, axis=0)

    # Generate embeddings for the "ANSWER" column
    embeddings = df[text_column].apply(lambda x: get_fasttext_embeddings(str(x)))

    # Stack embeddings into a 2D array
    embeddings = np.stack(embeddings.values)

    # Print the dimensions of the embeddings
    print(f"FastText embeddings dimensions for '{text_column}': {embeddings.shape}")

    # Return the embeddings and the original DataFrame
    return embeddings, df

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, flattening test_pred first
    # if it is a NumPy array
    test_pred_flat = test_pred.flatten() if isinstance(test_pred, np.ndarray) else test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred_flat)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred_flat, columns=all_labels).reindex(columns=all_labels, fill_value=0) # using flattened pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Define individual model functions
def knn_model(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")


def random_forest_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random Forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Navie Bayes")

def mlp_model(X_train, y_train, X_test, y_test):
    model = MLPClassifier(max_iter=300)
    model.fit(X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Mlp")

def xgboost_model(X_train, y_train, X_test, y_test):
    model = XGBClassifier(eval_metric='mlogloss')
    model.fit(X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG boost")

def adaboost_model(X_train, y_train, X_test, y_test):
    model = AdaBoostClassifier(algorithm='SAMME')
    model.fit(X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Adaboost")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=300)
    model.fit(X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    model = SVC()
    model.fit(X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Define RandomizedSearchCV for hyperparameter tuning for RandomForest
def random_forest_tune(X_train, y_train):
    param_dist = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': uniform(0.01, 0.1),
        'min_samples_leaf': uniform(0.01, 0.1)
    }
    rf = RandomForestClassifier()
    random_search = RandomizedSearchCV(rf, param_dist, n_iter=30, cv=5, verbose=1, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best RandomForest Parameters: {random_search.best_params_}")
    return random_search.best_estimator_

# Define a master function to call all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }

    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input and output file paths
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Specify the text column for FastText embeddings
text_column = 'answer'

# Convert text column to FastText embeddings and get the original DataFrame
embeddings, original_data = convert_to_fasttext_embeddings(input_file, text_column)

# Modify y to make sure the labels start from 0 (shift -1 to 0)
y = original_data['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.3, random_state=42)

# Run hyperparameter tuning for Random Forest with RandomizedSearchCV
best_rf_model = random_forest_tune(X_train, y_train)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2024-11-25 16:16:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.68, 3.171.22.33, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz.2’

cc.en.300.bin.gz.2  100%[===================>]   4.19G  92.6MB/s    in 38s     

2024-11-25 16:17:13 (112 MB/s) - ‘cc.en.300.bin.gz.2’ saved [4503593528/4503593528]

gzip: cc.en.300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten
FastText embeddings dimensions for 'answer': (681, 300)
Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best RandomForest Parameters: {'max_depth': 30, 'min_samples_leaf': 0.0

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Model Metrics:
Train Accuracy: 0.9517
Train Precision: 0.9521
Train Recall: 0.9517
Train F1 Score: 0.9509
Train Confusion Matrix:
[[ 27   4   1]
 [  0 117  15]
 [  0   3 309]]

Test Accuracy: 0.6683
Test Precision: 0.6768
Test Recall: 0.6683
Test F1 Score: 0.6708
Test Confusion Matrix:
[[  0   8   5]
 [  5  17  19]
 [  2  29 120]]


Performance Table for Mlp:
Metric              Value     
------------------------------
Train Accuracy      95.17%
Test Accuracy       66.83%
Train-Test Gap      28.34%
ROC-AUC             0.65
Overfitting         Yes


Running XGBoost model...
XGBoost Model Metrics:
Train Accuracy: 0.9937
Train Precision: 0.9937
Train Recall: 0.9937
Train F1 Score: 0.9937
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   1 311]]

Test Accuracy: 0.6976
Test Precision: 0.6403
Test Recall: 0.6976
Test F1 Score: 0.6668
Test Confusion Matrix:
[[  0   7   6]
 [  2  10  29]
 [  0  18 133]]


Performance Table for XG boost:
Metric              Value     
---------

In [3]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import fasttext  # fastText import
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert text column to fastText embeddings
def convert_to_fasttext(input_file, text_column, model_path=None):
    df = pd.read_excel(input_file)

    # Load fastText model if provided (otherwise you can use pretrained)
    if model_path:
        model = fasttext.load_model(model_path)
    else:
        model = fasttext.load_model('cc.en.300.bin')  # Pre-trained model (English)

    # Get fastText embeddings for each text in the column
    def get_fasttext_embedding(text):
        words = text.split()
        word_vectors = np.array([model.get_word_vector(word) for word in words if word in model])
        return np.mean(word_vectors, axis=0) if len(word_vectors) > 0 else np.zeros(300)  # Size of embeddings

    # Apply fastText embeddings to the text column
    df['fasttext_embeddings'] = df[text_column].apply(lambda x: get_fasttext_embedding(str(x)))

    # Convert the embeddings into a DataFrame
    embeddings = np.array(df['fasttext_embeddings'].to_list())
    return pd.DataFrame(embeddings), df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca
    # Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred, flattening test_pred first
    # if it is a NumPy array
    test_pred_flat = test_pred.flatten() if isinstance(test_pred, np.ndarray) else test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred_flat)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred_flat, columns=all_labels).reindex(columns=all_labels, fill_value=0) # using flattened pred
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")



def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Cataboost")


# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'

# Convert text column to fastText embeddings and get the original DataFrame
fasttext_data, original_data = convert_to_fasttext(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(fasttext_data, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 53
Running KNN model...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Params: {'weights': 'distance', 'n_neighbors': 7}
KNN Model Metrics:
Train Accuracy: 0.9748
Train Precision: 0.9754
Train Recall: 0.9748
Train F1 Score: 0.9746
Train Confusion Matrix:
[[ 31   0   1]
 [  0 122  10]
 [  0   1 311]]

Test Accuracy: 0.7024
Test Precision: 0.6441
Test Recall: 0.7024
Test F1 Score: 0.6620
Test Confusion Matrix:
[[  1   0  12]
 [  2   8  31]
 [  1  15 135]]


Performance Table for Cataboost:
Metric              Value     
------------------------------
Train Accuracy      97.48%
Test Accuracy       70.24%
Train-Test Gap      27.24%
ROC-AUC             0.55
Overfitting         Yes


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9727
Train Precision: 0.9731
Train Recall: 0.9727
Train F1 Score: 0.9724
Train Confusion Matrix:
[[ 30   1   1]
 [  0 122  10]
 [  0   1 311]]

Test Accuracy: 0.7073
Test Precision: 0.6272
Test Recall: 0.7073
Test F1 Score: 0.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8214
Train Precision: 0.8273
Train Recall: 0.8214
Train F1 Score: 0.8085
Train Confusion Matrix:
[[ 18   5   9]
 [  0  72  60]
 [  0  11 301]]

Test Accuracy: 0.7024
Test Precision: 0.6698
Test Recall: 0.7024
Test F1 Score: 0.6831
Test Confusion Matrix:
[[  1   2  10]
 [  2  14  25]
 [  2  20 129]]


Performance Table for Cataboost:
Metric              Value     
------------------------------
Train Accuracy      82.14%
Test Accuracy       70.24%
Train-Test Gap      11.90%
ROC-AUC             0.60
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6341
Test Precision: 0.6148
Test Recall: 0.6341


...

Best Params: {'learning_rate': 0.3, 'iterations': 50, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.9181
Train Precision: 0.9230
Train Recall: 0.9181
Train F1 Score: 0.9139
Train Confusion Matrix:
[[ 19   4   9]
 [  0 107  25]
 [  0   1 311]]

Test Accuracy: 0.7073
Test Precision: 0.6462
Test Recall: 0.7073
Test F1 Score: 0.6742
Test Confusion Matrix:
[[  0   3  10]
 [  0  13  28]
 [  0  19 132]]


Performance Table for Cataboost:
Metric              Value     
------------------------------
Train Accuracy      91.81%
Test Accuracy       70.73%
Train-Test Gap      21.08%
ROC-AUC             0.58
Overfitting         Yes


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.6912
Train Precision: 0.6829
Train Recall: 0.6912
Train F1 Score: 0.6383
Train Confusion Matrix:
[[  6   5  21]
 [  0  30 102]
 [  0  19 293]]

Test Accuracy: 0.6927
Test Precision: 0.6154
Test Recall: 0.6927
Test F1 Score: 0.6467
Test Confusion Matrix:
[[  0   1  12]
 [  0   8  33]
 [  1  16 134]]


Performance Table for Cataboost:
Metric              Value     
------------------------------
Train Accuracy      69.12%
Test Accuracy       69.27%
Train-Test Gap      0.15%
ROC-AUC             0.53
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8151
Train Precision: 0.8256
Train Recall: 0.8151
Train F1 Score: 0.7958
Train Confusion Matrix:
[[ 19   6   7]
 [  1  62  69]
 [  2   3 307]]

Test Accuracy: 0.6878
Test Precision: 0.6431
Test Recall: 0.6878
Test F1 Score: 0.6599
Test Confusion Matrix:
[[  1   1  11]
 [  1  10  30]
 [  5  16 130]]


Performance Table for Cataboost:
Metric              Value     
------------------------------
Train Accuracy      81.51%
Test Accuracy       68.78%
Train-Test Gap      12.73%
ROC-AUC             0.55
Overfitting         Yes


